In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
VIEWED_NOTIFICATIONS_PATH = '../data/transformed/hub_notifications_logs_transformed.csv'
DATA_PATH = '../data/transformed/notifications_users.csv'
JOINED_DATA_PATH = '../data/transformed/Combined_views.csv'
# Load transformed notifications data
df_notifications = pd.read_csv(VIEWED_NOTIFICATIONS_PATH)

# print(f"Loaded notifications data: {df_notifications.shape[0]} records with {df_notifications.shape[1]} columns")
# df_notifications.head()

In [3]:
# Load Combined data 
df_data = pd.read_csv(DATA_PATH)

In [4]:
columns_to_drop = ['id_notificationlogs', 'capstone_employee_id_notificationlogs', 'capstone_name_notificationlogs']
df_notifications = df_notifications.drop(columns=columns_to_drop)

columns1_to_drop = ['id', 'time_diff_days', 'tile_description','jobtitle']
df_data = df_data.drop(columns=columns1_to_drop)

In [5]:
import pandas as pd
import os

# Step 1: Normalize emails
df_notifications["capstone_email_notificationlogs"] = df_notifications["capstone_email_notificationlogs"].str.strip().str.lower()
df_data["capstone_email"] = df_data["capstone_email"].str.strip().str.lower()

# Step 2: Rename email columns for merge
df_notifications = df_notifications.rename(columns={"capstone_email_notificationlogs": "email"})
df_data = df_data.rename(columns={"capstone_email": "email"})

# Step 3: Convert time columns to datetime
df_notifications["View_time"] = pd.to_datetime(df_notifications["View_time"])
df_data["start"] = pd.to_datetime(df_data["start"])
df_data["end"] = pd.to_datetime(df_data["end"])

# Step 4: Left merge to keep all View_time records
merged_df = df_notifications.merge(
    df_data,
    left_on="email",
    right_on="email",
    how="left"
)

# Step 5: Mark valid records where View_time falls between start and end
merged_df["valid"] = (merged_df["View_time"] >= merged_df["start"]) & \
                     (merged_df["View_time"] <= merged_df["end"])

# Step 6: Keep rows that matched or have no start time (unmatched)
filtered_df = merged_df[merged_df["valid"] | merged_df["start"].isna()]

# Step 7: Group by user and View_time
grouped_df = filtered_df.groupby(["email", "View_time"]).agg(list).reset_index()

# Step 8: Add count of start-end pairs
grouped_df["count"] = grouped_df["start"].apply(lambda x: len(x))

# Step 9: Expand start and end into separate columns
final_df = grouped_df.copy()
max_pairs = grouped_df["start"].apply(len).max()

for i in range(max_pairs):
    final_df[f"start_{i+1}"] = grouped_df["start"].apply(lambda x: x[i] if i < len(x) else "")
    final_df[f"end_{i+1}"] = grouped_df["end"].apply(lambda x: x[i] if i < len(x) else "")

# Step 10: Drop original start and end list columns
final_df = final_df.drop(columns=["start", "end"])

# Step 11: Convert metadata columns to comma-separated strings
metadata_columns = ["title", "description", "notification_type", "tile_id", "tile_name", "tile_source", "tile_roles"]
for col in metadata_columns:
    if col in final_df.columns:
        final_df[col] = final_df[col].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else x)

# Step 12: Fill NaNs with empty string
final_df = final_df.fillna("")

# Display preview
#print(final_df.head())

C:\Users\HOME\AppData\Local\Temp\ipykernel_18228\2168549126.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[f"end_{i+1}"] = grouped_df["end"].apply(lambda x: x[i] if i < len(x) else "")
C:\Users\HOME\AppData\Local\Temp\ipykernel_18228\2168549126.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df[f"start_{i+1}"] = grouped_df["start"].apply(lambda x: x[i] if i < len(x) else "")
C:\Users\HOME\AppData\Local\Temp\ipykernel_18228\2168549126.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [6]:
# Step 9: Save to CSV
os.makedirs(os.path.dirname(JOINED_DATA_PATH), exist_ok=True)
final_df.to_csv(JOINED_DATA_PATH, index=False)
print(f'Joined data saved to {JOINED_DATA_PATH}')

Joined data saved to ../data/transformed/Combined_views.csv
